In [1]:
import pandas as pd   
from sklearn.model_selection import train_test_split
import yfinance as yf
import pandas_ta as ta  
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import pickle 
import os
from datetime import datetime 

In [3]:

crypto = ['BTC', 'ETH', 'SOL', 'USDT', 'BNB']

In [15]:
data = pd.read_csv('BTC.csv')

In [16]:
data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,0.0,0.0
1,2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,0.0,0.0
2,2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,0.0,0.0
3,2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,0.0,0.0
4,2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,0.0,0.0
...,...,...,...,...,...,...,...,...
3629,2024-08-24 00:00:00+00:00,64103.871094,64513.789062,63619.917969,64178.992188,21430585163,0.0,0.0
3630,2024-08-25 00:00:00+00:00,64176.367188,64996.421875,63833.519531,64333.542969,18827683555,0.0,0.0
3631,2024-08-26 00:00:00+00:00,64342.226562,64489.707031,62849.558594,62880.660156,27682040631,0.0,0.0
3632,2024-08-27 00:00:00+00:00,62879.707031,63210.796875,58116.750000,59504.132812,39103882198,0.0,0.0


In [17]:
data.drop([ 'Open', 'High', 'Low',  'Dividends', 'Stock Splits', 'Volume'], inplace = True, axis = 1)
data

,Date,Close
0,2014-09-17 00:00:00+00:00,457.334015
1,2014-09-18 00:00:00+00:00,424.440002
2,2014-09-19 00:00:00+00:00,394.795990
3,2014-09-20 00:00:00+00:00,408.903992
4,2014-09-21 00:00:00+00:00,398.821014
...,...,...
3629,2024-08-24 00:00:00+00:00,64178.992188
3630,2024-08-25 00:00:00+00:00,64333.542969
3631,2024-08-26 00:00:00+00:00,62880.660156
3632,2024-08-27 00:00:00+00:00,59504.132812


In [19]:
data['time'] = pd.to_datetime(data['Date'])

In [20]:
data['time']

0      2014-09-17 00:00:00+00:00
1      2014-09-18 00:00:00+00:00
2      2014-09-19 00:00:00+00:00
3      2014-09-20 00:00:00+00:00
4      2014-09-21 00:00:00+00:00
                  ...           
3629   2024-08-24 00:00:00+00:00
3630   2024-08-25 00:00:00+00:00
3631   2024-08-26 00:00:00+00:00
3632   2024-08-27 00:00:00+00:00
3633   2024-08-28 00:00:00+00:00
Name: time, Length: 3634, dtype: datetime64[ns, UTC]

In [21]:
data['formatted_time'] = data['time'].dt.strftime('%d-%m-%Y') 

In [22]:
data

,Date,Close,time,formatted_time
0,2014-09-17 00:00:00+00:00,457.334015,2014-09-17 00:00:00+00:00,17-09-2014
1,2014-09-18 00:00:00+00:00,424.440002,2014-09-18 00:00:00+00:00,18-09-2014
2,2014-09-19 00:00:00+00:00,394.795990,2014-09-19 00:00:00+00:00,19-09-2014
3,2014-09-20 00:00:00+00:00,408.903992,2014-09-20 00:00:00+00:00,20-09-2014
4,2014-09-21 00:00:00+00:00,398.821014,2014-09-21 00:00:00+00:00,21-09-2014
...,...,...,...,...
3629,2024-08-24 00:00:00+00:00,64178.992188,2024-08-24 00:00:00+00:00,24-08-2024
3630,2024-08-25 00:00:00+00:00,64333.542969,2024-08-25 00:00:00+00:00,25-08-2024
3631,2024-08-26 00:00:00+00:00,62880.660156,2024-08-26 00:00:00+00:00,26-08-2024
3632,2024-08-27 00:00:00+00:00,59504.132812,2024-08-27 00:00:00+00:00,27-08-2024


In [23]:
data.drop([ 'Date', 'time'], inplace = True, axis = 1)

In [24]:
data['SMA'] = ta.sma(data['Close'], length=5) 
data['EMA'] = ta.ema(data['Close'], length=5) 
data['RSI'] = ta.rsi(data['Close'], length=14) 

In [25]:
data

,Close,formatted_time,SMA,EMA,RSI
0,457.334015,17-09-2014,NaN,NaN,NaN
1,424.440002,18-09-2014,NaN,NaN,NaN
2,394.795990,19-09-2014,NaN,NaN,NaN
3,408.903992,20-09-2014,NaN,NaN,NaN
4,398.821014,21-09-2014,416.859003,416.859003,NaN
...,...,...,...,...,...
3629,64178.992188,24-08-2024,61768.649219,62288.975623,58.584205
3630,64333.542969,25-08-2024,62832.799219,62970.498072,58.922064
3631,62880.660156,26-08-2024,63173.892969,62940.552100,54.427081
3632,59504.132812,27-08-2024,62998.336719,61795.079004,45.701321


In [30]:
data.fillna(0, inplace = True)

In [32]:
X = data.drop(['Close', 'formatted_time'], axis = 1).values
X

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.31738930e+04, 6.29405521e+04, 5.44270808e+01],
       [6.29983367e+04, 6.17950790e+04, 4.57013213e+01],
       [6.21656047e+04, 6.11736178e+04, 4.68603756e+01]])

In [33]:
Y = data['Close'].values

In [34]:
Y

array([  457.33401489,   424.44000244,   394.79598999, ...,
       62880.66015625, 59504.1328125 , 59930.6953125 ])

In [35]:
rf = RandomForestRegressor()

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [38]:
param_grid = {
        'n_estimators': [50, 100, 150, 200, 250, 300]
    }
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3)

In [39]:
grid_search_rf

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [50, 100, 150, 200, 250, 300]})

In [40]:
grid_search_rf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'n_estimators': [50, 100, 150, 200, 250, 300]})

In [5]:
for type in crypto:
    if os.path.exists(type + '.csv'):
        os.remove(type + '.csv')
    data_ticker = yf.Ticker(type +"-USD")
    data = data_ticker.history(period="max")
    data.to_csv(type+".csv")
    data = pd.read_csv(type + '.csv')
    data.drop([ 'Open', 'High', 'Low',  'Dividends', 'Stock Splits', 'Volume'], inplace = True, axis = 1)
    data['time'] = pd.to_datetime(data['Date'])
    data['formatted_time'] = data['time'].dt.strftime('%d-%m-%Y') 
    data.drop([ 'Date', 'time'], inplace = True, axis = 1)
    data['SMA'] = ta.sma(data['Close'], length=5) 
    data['EMA'] = ta.ema(data['Close'], length=5) 
    data['RSI'] = ta.rsi(data['Close'], length=14) 
    data.fillna(0, inplace = True)
    X = data.drop(['Close', 'formatted_time'], axis = 1).values
    Y = data['Close'].values
    rf = RandomForestRegressor()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    param_grid = {
        'n_estimators': [50, 100, 150, 200, 250, 300]
    }
    grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3)
    grid_search_rf.fit(X_train, y_train)
    if os.path.exists(type + '.pkl'):
        os.remove(type + '.pkl')
    with open(type +'.pkl', 'wb') as f:
        pickle.dump(grid_search_rf.best_estimator_, f) 
        print(f"The model of {type} is successfully trained and the {type}.csv is updated and new {type}.pkl is formed at {datetime.now()}")


KeyboardInterrupt: 